# RNN for sentiment analysis
RNN can use context from sequential data. This can be used to compute sentiment of sentnce. In this article we will be apply RNNs to analyze the sentiment of customer reviews from Swiggy food delivery platform. The goal is to classify reviews as positive or negative for providing insights into customer experiences.

In [10]:
import pandas as pd
import numpy as np
import re  
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

In [19]:
df = pd.read_csv('swiggy.csv')
print("Columns in the dataset:")
print(df.columns.tolist())
print("####################")
print(df.head(5))

Columns in the dataset:
['ID', 'Area', 'City', 'Restaurant Price', 'Avg Rating', 'Total Rating', 'Food Item', 'Food Type', 'Delivery Time', 'Review']
####################
   ID               Area       City  Restaurant Price  Avg Rating  \
0   1             Suburb  Ahmedabad               600         4.2   
1   2  Business District       Pune               200         4.7   
2   3             Suburb  Bangalore               600         4.7   
3   4  Business District     Mumbai               900         4.0   
4   5          Tech Park     Mumbai               200         4.7   

   Total Rating        Food Item       Food Type Delivery Time  \
0          6198            Sushi       Fast Food     30-40 min   
1          4865  Pepperoni Pizza  Non-Vegetarian     50-60 min   
2          2095          Waffles       Fast Food     50-60 min   
3          6639            Sushi      Vegetarian     50-60 min   
4          6926     Spring Rolls     Gluten-Free     20-30 min   

                 

In [27]:
df["Review"] = df["Review"].str.lower()
df["Review"] = df["Review"].replace(r'[^a-z0-9\s]', '', regex=True)

df['sentiment'] = df['Avg Rating'].apply(lambda x: 1 if x > 3.5 else 0) # Uses Avg Rating to generate binary labels (positive if rating >3.5)
df = df.dropna() 

print(df.head(5))

   ID               Area       City  Restaurant Price  Avg Rating  \
0   1             Suburb  Ahmedabad               600         4.2   
1   2  Business District       Pune               200         4.7   
2   3             Suburb  Bangalore               600         4.7   
3   4  Business District     Mumbai               900         4.0   
4   5          Tech Park     Mumbai               200         4.7   

   Total Rating        Food Item       Food Type Delivery Time  \
0          6198            Sushi       Fast Food     30-40 min   
1          4865  Pepperoni Pizza  Non-Vegetarian     50-60 min   
2          2095          Waffles       Fast Food     50-60 min   
3          6639            Sushi      Vegetarian     50-60 min   
4          6926     Spring Rolls     Gluten-Free     20-30 min   

                           Review  sentiment  
0  good but nothing extraordinary          1  
1  good but nothing extraordinary          1  
2         late delivery ruined it          1  


In [28]:
max_features = 5000  
max_length = 200    

# Tokenizer: Converts words into integer sequences.
tokenizer = Tokenizer(num_words=max_features) 
tokenizer.fit_on_texts(df["Review"])

# Padding: Ensures all input sequences have the same length (max_length).
X = pad_sequences(tokenizer.texts_to_sequences(df["Review"]), maxlen=max_length)

y = df['sentiment'].values  

In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

In [30]:
# Embedding Layer: Converts integer sequences into dense vectors (16 dimensions).
# RNN Layer: Processes sequence data with 64 units and tanh activation.
# Output Layer: Predicts sentiment probability using sigmoid activation.

model = Sequential([
    Embedding(input_dim=max_features, output_dim=16, input_length=max_length),
    SimpleRNN(64, activation='tanh', return_sequences=False),                   
    Dense(1, activation='sigmoid')                                          
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


c:\Users\hi\Desktop\projects\python_projects\tutorial\tut_tensorflow\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [31]:
EPOCHS = 5
import time
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.times = []
    
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_time_start = time.time()
    
    def on_epoch_end(self, epoch, logs=None):
        self.times.append(time.time() - self.epoch_time_start)

# Create callback
time_callback = TimeHistory()


history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1,
    callbacks=[time_callback]
)

# After training, you can see time per epoch
print("\nTime per epoch (in seconds):", time_callback.times)

# Total training time in minutes
total_time_min = sum(time_callback.times) / 60
print(f"Total training time: {total_time_min:.2f} minutes")


Epoch 1/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.6807 - loss: 0.6185 - val_accuracy: 0.7156 - val_loss: 0.5981
Epoch 2/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.7204 - loss: 0.5934 - val_accuracy: 0.7156 - val_loss: 0.5961
Epoch 3/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.7207 - loss: 0.5935 - val_accuracy: 0.7156 - val_loss: 0.5979
Epoch 4/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.7122 - loss: 0.6018 - val_accuracy: 0.7156 - val_loss: 0.5976
Epoch 5/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.7184 - loss: 0.5926 - val_accuracy: 0.7156 - val_loss: 0.5961

Time per epoch (in seconds): [7.168249607086182, 4.835086822509766, 4.196996450424194, 4.187966585159302, 4.166996717453003]
Total training time: 0.41 minutes


In [42]:
score = model.evaluate(X_test, y_test)
print(f"Test accuracy: {score[1]:.2f}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7027 - loss: 0.6091
Test accuracy: 0.72
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
[[ 455    0]
 [1145    0]]


In [39]:
print(score)

[0.597517192363739, 0.715624988079071]


In [36]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"
    

sample_review = "The food was great."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")


sample_review = "The food was not great"
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")


Review: The food was not great
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Sentiment: Positive (Probability: 0.71)
